
# Phase synchrony lattices

## Objectives
- Illustrate how local lattices distribute phase adjustments to converge toward a shared synchrony ridge.
- Document the deterministic ΔNFR hook required by Phase-2 lattice controllers to coordinate νf, phase, and EPI adjustments.
- Provide a measurable synchrony lift that downstream CI can assert without stochastic noise.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the lattice coordination contract mirrors the staged deployment plan.
- :mod:`tnfr.dynamics.coordination` — the future lattice supervisor reuses these helpers to compute mean phase envelopes.
- :mod:`tnfr.observers` — `phase_sync` is the agreed telemetry primitive for the lattice health dashboards.

## Theoretical exposition
A phase synchrony lattice binds neighbouring nodes by iteratively nudging their phases toward a common anchor while respecting ΔNFR scaling. Each operator pulse captures the global mean phase, applies a weighted correction, and records the induced ΔNFR so the nodal equation can update EPI consistently. Because synchrony depends on the dispersion of θ values, even small deterministic corrections shrink the variance and raise the Kuramoto-inspired synchrony index exported by :func:`tnfr.observers.phase_sync`.

## Deterministic smoke check
The example constructs a star-shaped lattice, records its initial synchrony, and then applies a scripted hook while running the canonical emission→reception→coherence→resonance→transition segment. The hook logs the synchrony index after every glyph so the trace reveals a monotonic lift, confirming that ΔNFR bookkeeping and phase adjustments remain coherent.


In [ ]:

from statistics import mean

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, THETA_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.observers import phase_sync
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence

G, anchor = create_nfr("lattice-anchor", epi=0.35, vf=1.1, theta=0.1)
neighbors: list[str] = []
for idx, offset in enumerate([0.55, -0.42, 0.23, -0.31], start=1):
    _, node = create_nfr(f"lattice-node-{idx}", epi=0.28 + 0.02 * idx, vf=0.95 + 0.01 * idx, theta=offset, graph=G)
    G.add_edge(anchor, node)
    neighbors.append(node)

before_phases = {n: round(float(G.nodes[n][THETA_PRIMARY]), 6) for n in [anchor, *neighbors]}
initial_sync = round(phase_sync(G), 6)
synchrony_trace: list[dict[str, float]] = []


def align_lattice(graph):
    phases = [float(data[THETA_PRIMARY]) for _, data in graph.nodes(data=True)]
    mean_phase = mean(phases)
    for node_id, data in graph.nodes(data=True):
        theta = float(data[THETA_PRIMARY])
        offset = mean_phase - theta
        data[DNFR_PRIMARY] = abs(offset) * 0.04
        data[THETA_PRIMARY] = theta + 0.6 * offset
        vf = float(data[VF_PRIMARY])
        epi = float(data[EPI_PRIMARY])
        data[EPI_PRIMARY] = epi + vf * data[DNFR_PRIMARY] * 0.5
    synchrony_trace.append({
        "step": len(synchrony_trace) + 1,
        "phase_sync": round(phase_sync(graph), 6),
    })

set_delta_nfr_hook(G, align_lattice, note="phase lattice synchrony smoke")
run_sequence(G, anchor, [Emission(), Reception(), Coherence(), Resonance(), Transition()])

after_phases = {n: round(float(G.nodes[n][THETA_PRIMARY]), 6) for n in [anchor, *neighbors]}
final_sync = round(phase_sync(G), 6)

{
    "initial_sync": initial_sync,
    "final_sync": final_sync,
    "before_phases": before_phases,
    "after_phases": after_phases,
    "synchrony_trace": synchrony_trace,
}
